In [ ]:
import requests
from bs4 import BeautifulSoup
from itertools import cycle

'''
file = open('http_proxies.csv')
d = file.read()
proxy_list = d.split('\n')
'''
proxy_list = [] # List of proxy ips
profile_url = 'https://twitter.com/sachin_rt' # Twitter profile URL
proxy_cycle = cycle(proxy_list) # Create a cycle of proxy IPs
data = {} # Initialize variables to store scraped data
session = requests.Session() # Set up a session with requests

def rotate_proxy(): # Function to rotate proxy IPs
    proxy = next(proxy_cycle)
    session.proxies = {'http': proxy, 'https': proxy}

def scrape_profile_data(url): # Function to scrape profile data
    try:
        response = session.get(url)  # Make a request using the current proxy IP
        soup = BeautifulSoup(response.content, 'html.parser')  # Parse the HTML content using Beautiful Soup
        # Scrape required data from the profile page
        data['biography'] = soup.select_one('meta[name="description"]')['content']
        data['followers_count'] = int(soup.select_one('a[href="/sachin_rt/followers"] span').text.replace(',', ''))
        data['following_count'] = int(soup.select_one('a[href="/sachin_rt/following"] span').text.replace(',', ''))
        data['likes_count'] = int(soup.select_one('a[href="/sachin_rt/likes"] span').text.replace(',', ''))
        data['user_id'] = int(soup.select_one('div[data-testid="primaryColumn"]')['data-legacy-profile-id'])
        print(data) # Print the scraped data
    except:
        print("Request failed. Rotating proxy and retrying...") # If request fails, rotate proxy and retry
        rotate_proxy()
        scrape_profile_data(url)

scrape_profile_data(profile_url) # Start scraping the profile data